In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
path='C:/Users/mehak/OneDrive - University of Delaware - o365/Beheshti, Rahmat - Mehak - Brennan/data/'

In [2]:
data=pd.read_csv(path+'day_intervals/cohort/cohort30day.csv.gz', compression='gzip', header=0, index_col=None)

In [3]:
from datetime import datetime

data['admittime'] = pd.to_datetime(data['admittime'])
data['dischtime'] = pd.to_datetime(data['dischtime'])

In [4]:
data.head()

,admittime,dischtime,hadm_id,min_valid_year,subject_id,label
0,2180-06-26 18:27:00,2180-06-27 18:49:00,22841357.0,2183.0,10000032.0,1
1,2180-07-23 12:35:00,2180-07-25 17:55:00,29079034.0,2183.0,10000032.0,1
2,2146-12-05 19:07:00,2146-12-12 16:30:00,20032235.0,2155.0,10000826.0,1
3,2146-12-18 17:39:00,2146-12-24 19:55:00,21086876.0,2155.0,10000826.0,1
4,2124-05-14 17:25:00,2124-05-14 21:10:00,29957930.0,2133.0,10000883.0,1


In [5]:
data['los']=pd.to_timedelta(data['dischtime']-data['admittime'],unit='h')
data['los']=data['los'].astype(str)

In [6]:
data[['days', 'dummy','hours']] = data['los'].str.split(' ', -1, expand=True)
data[['hours','min','sec']] = data['hours'].str.split(':', -1, expand=True)
data['los']=pd.to_numeric(data['days'])*24+pd.to_numeric(data['hours'])

In [7]:
data=data.drop(columns=['days', 'dummy','hours','min','sec'])

In [8]:
data.head()

,admittime,dischtime,hadm_id,min_valid_year,subject_id,label,los
0,2180-06-26 18:27:00,2180-06-27 18:49:00,22841357.0,2183.0,10000032.0,1,24
1,2180-07-23 12:35:00,2180-07-25 17:55:00,29079034.0,2183.0,10000032.0,1,53
2,2146-12-05 19:07:00,2146-12-12 16:30:00,20032235.0,2155.0,10000826.0,1,165
3,2146-12-18 17:39:00,2146-12-24 19:55:00,21086876.0,2155.0,10000826.0,1,146
4,2124-05-14 17:25:00,2124-05-14 21:10:00,29957930.0,2133.0,10000883.0,1,3


In [9]:
data=data[data['los']>0]

In [10]:
data['los'].min()

1

In [11]:
data['los'].max()

11210

In [12]:
data['subject_id'].nunique()

189382

In [13]:
data[data['label']==1]['hadm_id'].nunique()

88015

In [14]:
data[data['label']==0]['hadm_id'].nunique()

359625

In [15]:
data['hadm_id'].nunique()

447640

In [12]:
cond=pd.read_csv(path+'long_format/diag/long_diag_icd10_roots_norm.csv.gz', compression='gzip', header=0, index_col=None)

In [13]:
cond.head()

,subject_id,hadm_id,root,timedelta
0,15734973,20475282,D58,0
1,15734973,20012083,O44,1798
2,15734973,20012083,D58,1798
3,11442057,21518990,O61,0
4,11442057,21518990,O10,0


In [14]:
cond=cond[cond['hadm_id'].isin(data['hadm_id'])]

In [15]:
cond['subject_id'].nunique()

139350

In [16]:
cond['hadm_id'].nunique()

296593

In [17]:
meds=pd.read_csv(path+'long_format/meds/preproc_med_nonproprietary.csv.gz', compression='gzip', header=0, index_col=None)

In [18]:
meds.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit
0,10000032,22595853,2180-05-07 01:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 02:37:00,0 days 23:37:00
1,10000032,22595853,2180-05-07 01:00:00,furosemide,0 days 02:37:00,0 days 10:37:00
2,10000032,22841357,2180-06-26 23:00:00,furosemide,0 days 04:33:00,0 days 03:33:00
3,10000032,25742920,2180-08-07 08:00:00,furosemide,1 days 08:16:00,1 days 22:16:00
4,10000032,29079034,2180-07-23 17:00:00,fluticasone propionate,0 days 04:25:00,2 days 09:25:00


In [19]:
meds[['start_days', 'dummy','start_hours']] = meds['start_hours_from_admit'].str.split(' ', -1, expand=True)
meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
meds['start_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])

In [20]:
meds[['start_days', 'dummy','start_hours']] = meds['stop_hours_from_admit'].str.split(' ', -1, expand=True)
meds[['start_hours','min','sec']] = meds['start_hours'].str.split(':', -1, expand=True)
meds['stop_time']=pd.to_numeric(meds['start_days'])*24+pd.to_numeric(meds['start_hours'])

In [21]:
meds=meds.drop(columns=['start_days', 'dummy','start_hours','min','sec'])

In [22]:
meds['sanity']=meds['stop_time']-meds['start_time']
meds=meds[meds['sanity']>0]
del meds['sanity']

In [23]:
meds.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,start_time,stop_time
0,10000032,22595853,2180-05-07 01:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 02:37:00,0 days 23:37:00,2,23
1,10000032,22595853,2180-05-07 01:00:00,furosemide,0 days 02:37:00,0 days 10:37:00,2,10
3,10000032,25742920,2180-08-07 08:00:00,furosemide,1 days 08:16:00,1 days 22:16:00,32,46
4,10000032,29079034,2180-07-23 17:00:00,fluticasone propionate,0 days 04:25:00,2 days 09:25:00,4,57
5,10000032,25742920,2180-08-06 03:00:00,fluticasone propionate,0 days 03:16:00,1 days 22:16:00,3,46


In [24]:
meds=meds[meds['hadm_id'].isin(data['hadm_id'])]

In [25]:
meds=pd.merge(meds,data[['hadm_id','los']],on='hadm_id',how='left')

In [26]:
meds['sanity']=meds['los']-meds['start_time']
meds=meds[meds['sanity']>0]
del meds['sanity']
meds.loc[meds['stop_time'] > meds['los'],'stop_time']=meds.loc[meds['stop_time'] > meds['los'],'los']
del meds['los']

In [27]:
meds['subject_id'].nunique()

123977

In [28]:
meds['hadm_id'].nunique()

255932

In [21]:
proc=pd.read_csv(path+'long_format/proc/long_proc_icd10_norm.csv.gz', compression='gzip', header=0, index_col=None)

In [22]:
proc.head()

,subject_id,hadm_id,icd_code,timedelta
0,10001401,21544441,0UT9FZZ,0
1,10001401,21544441,07BC4ZX,0
2,10001401,21544441,0TTB4ZZ,0
3,10001401,21544441,8E0W4CZ,0
4,10001401,21544441,0T1807C,0


In [23]:
proc=proc[proc['hadm_id'].isin(data['hadm_id'])]

In [24]:
proc['subject_id'].nunique()

10810

In [25]:
proc['hadm_id'].nunique()

13555

In [26]:
labs=pd.read_csv(path+'long_format/labs/long_labs_units_cleaned_norm.csv.gz', compression='gzip', header=0, index_col=None)

In [27]:
labs.head()

,subject_id,hadm_id,valuenum,timedelta,itemid
0,10000032,22841357.0,15.7,95,51277
1,10000032,NaN,16.2,121,51274
2,10000032,NaN,32.4,121,51275
3,10000032,NaN,153.0,121,50861
4,10000032,NaN,3.6,121,50862


In [28]:
labs.shape

(53180040, 5)

In [29]:
labs=labs.dropna()

In [66]:
labs.isna().sum()

subject_id    0
hadm_id       0
valuenum      0
timedelta     0
valuenum.1    0
dtype: int64

In [67]:
21636266/53180040

0.4068493743141224

In [30]:
labs=labs[labs['hadm_id'].isin(data['hadm_id'])]

In [31]:
labs['subject_id'].nunique()

114824

In [32]:
labs['hadm_id'].nunique()

239156

In [ ]:
########MAx Min Length

In [153]:
min_length=2
max_length=72

In [154]:
data=data[(data['los']>=min_length) & (data['los']<=max_length)]

In [155]:
########Select time duration needed

In [29]:
include_time=24

In [30]:
data=data[(data['los']>=include_time)]
meds=meds[meds['hadm_id'].isin(data['hadm_id'])]

In [32]:
cond=cond[cond['hadm_id'].isin(data['hadm_id'])]

In [31]:
data['select_time']=data['los']-include_time
data['los']=include_time

In [ ]:
proc=pd.merge(proc,data[['hadm_id','select_time']],on='hadm_id',how='left')
proc['start_time']=proc['start_time']-proc['select_time']
proc=proc[proc['start_time']>=0]

In [34]:
meds=pd.merge(meds,data[['hadm_id','select_time']],on='hadm_id',how='left')
meds['stop_time']=meds['stop_time']-meds['select_time']
meds['start_time']=meds['start_time']-meds['select_time']
meds=meds[meds['stop_time']>=0]
meds.loc[meds.start_time <0, 'start_time']=0

In [35]:
meds['start_time'].max()

23

In [ ]:
labs=pd.merge(labs,data[['hadm_id','select_time']],on='hadm_id',how='left')
labs['start_time']=labs['start_time']-labs['select_time']
labs=labs[labs['start_time']>=0]

In [ ]:
if stay_duration>0:
    cond['select_time']=stay_duration-exclude_time
    proc['select_time']=stay_duration-exclude_time
    meds['select_time']=stay_duration-exclude_time
    labs['select_time']=stay_duration-exclude_time
    cond=cond.loc[:,cond.loc['time'] < cond.loc['select_time']]
    proc=proc.loc[:,proc.loc['time'] < proc.loc['select_time']]
    meds=meds.loc[:,meds.loc['time'] < meds.loc['select_time']]
    labs=labs.loc[:,labs.loc['time'] < labs.loc['select_time']]
else:
    data['select_time']=data['los']-exclude_time
    data['los']=data['los']-exclude_time
    
    cond=pd.merge(cond,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    cond=cond.loc[:,cond.loc['time'] < cond.loc['select_time']]

    proc=pd.merge(proc,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    proc=proc.loc[:,proc.loc['time'] < proc.loc['select_time']]

    meds=pd.merge(meds,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    meds=meds.loc[:,meds.loc['start_time'] < meds.loc['select_time']]

    labs=pd.merge(labs,adm[['hadm_id','select_time']],on='hadm_id',how='left')
    labs=labs.loc[:,labs.loc['time'] < labs.loc['select_time']]
    

In [ ]:
#bucketing

In [66]:
bucket=2

In [37]:
####Create dictionary

In [38]:
hids=meds['hadm_id'].unique()

In [39]:
len(hids)

224745

In [40]:
dataDic={}
for hid in hids:
    subDic={}
    grp=data[data['hadm_id']==hid]
    #print(grp)
    dataDic[hid]={'Cond':{},'Proc':{},'Med':{},'label':int(grp['label'])}
    #print(condDic)

In [41]:
dataDic

{25742920: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 29079034: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 22841357: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 28979390: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 24558333: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 20823482: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 27897940: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 21086876: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 28289260: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 20032235: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 25221576: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 25849114: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 26381316: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 29541074: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 1},
 25242409: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 24947999: {'Cond': {}, 'Proc': {}, 'Med': {}, 'label': 0},
 25911675: {'Cond': {}, 'Proc': {}, 'Med

In [73]:
maxcount=0
for hid in hids:
    grp=cond[cond['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Cond']={'fids':list(grp['root'])}
    #print(condDic)

In [74]:
maxcount

56

In [ ]:
def pad(grp,final):#,labelsDead,labelsSurvived,pt):
    
    grp=grp.sort_values(by=['time'])
    if(non-lab):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
    if(meds):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
    
    if(proc):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            if(grp1.shape[0]==0):
                grp1['val']=0
            else:
                grp1['val']=1     
            grp1['new_time']=new_time
            final=final.append(grp1)
            dataDic[hid]['Proc'][grp['icd_code'][0]]=
            new_time=new_time+1
            
    if(lab):
        new_time=1
        for i in range(1,los,bucket):
            grp1=grp[(grp['time']>i) & (grp['time']<i+bucket)]
            grp1['new_time']=new_time
            final=final.append(grp1)
            new_time=new_time+1
      
    return final

In [ ]:
def full_signal(grp):
    grp=grp.sort_values(by=['time'])
    val=[]
    for i in range(1,los,bucket):
        grp1=grp[(grp['time']>=i) & (grp['time']<i+bucket)]
        if(grp1.shape[0]==0):
            val=val.append(0)
        else:
            val=val.append(1)

In [ ]:
maxcount=0
for hid in hids:
    sub=proc[proc['hadm_id']==hid]
    groups=sub.group_by('icd_code')
    for icd_code, grp in groups:
        sig=full_signal(grp)
        dataDic[hid]['Proc'][icd_code]=list(sig)

In [75]:
maxcount=0
for hid in hids:
    grp=proc[proc['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Proc']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Proc']={'fids':list(grp['icd_code'])}
    #print(condDic)

In [76]:
maxcount

40

In [42]:
los=24

In [43]:
meds=meds.sort_values(by=['start_time'])

In [49]:
sample=meds[meds['hadm_id'].isin(hids[0:3])]

In [50]:
sample.shape

(33, 9)

In [51]:
sample['hadm_id'].nunique()

3

In [52]:
sample.head()

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,start_time,stop_time,select_time
14,10000032,29079034,2180-07-23 17:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 04:25:00,2 days 09:25:00,0,24,29
15,10000032,25742920,2180-08-06 03:00:00,emtricitabine and tenofovir disoproxil fumarate,0 days 03:16:00,1 days 22:16:00,0,24,18
18,10000032,25742920,2180-08-06 03:00:00,rifaximin,0 days 03:16:00,1 days 22:16:00,0,24,18
17,10000032,29079034,2180-07-23 20:00:00,rifaximin,0 days 07:25:00,2 days 09:25:00,0,24,29
9,10000032,25742920,2180-08-06 03:00:00,raltegravir,0 days 03:16:00,1 days 22:16:00,0,24,18


In [221]:
def full_signal(grp):
    grp=grp.sort_values(by=['start_time'])
    val=[]
    on_med=0
    for i in range(0,los,bucket):
        start=grp[(grp['start_time']>=i) & (grp['start_time']<i+bucket)]
        stop=grp[(grp['stop_time']>=i) & (grp['stop_time']<i+bucket)]
        if(start.shape[0]==0):
            if(on_med==1):
                if(stop.shape[0]==0):
                    on_med=1
                else:
                    on_med=0
        else:
            on_med=1
        val.append(on_med)
    return val

In [92]:
los=24
bucket=2

In [93]:
final=pd.DataFrame()
t=0
for i in tqdm(range(0,los,bucket)): 
    sub=meds[(meds['start_time']>=i) & (meds['start_time']<i+bucket)].groupby(['hadm_id','nonproprietaryname']).agg({'stop_time':'max','subject_id':'max'})
    sub=sub.reset_index()
    sub['start_time']=t
    t=t+1
    sub['stop_time']=sub['stop_time']/bucket
    if final.empty:
        final=sub
    else:
        final=final.append(sub)

100%|██████████| 12/12 [00:01<00:00,  6.13it/s]


In [94]:
final.head()

,hadm_id,nonproprietaryname,stop_time,subject_id,start_time
0,20000019,acetaminophen,12.0,10467237,0
1,20000019,aspirin,12.0,10467237,0
2,20000019,ciprofloxacin hydrochloride,12.0,10467237,0
3,20000019,dextrose monohydrate,12.0,10467237,0
4,20000019,heparin sodium,12.0,10467237,0


In [95]:
f2=final.groupby(['hadm_id','nonproprietaryname']).size()
f2.groupby('hadm_id').sum().reset_index()[0].max()

37

In [96]:
final.groupby('hadm_id').size().max()

37

In [97]:
final.shape

(1722810, 5)

In [60]:
meds.shape

(1802806, 9)

In [393]:
sample['val']=1

In [63]:
import numpy as np

In [61]:
hids=final['hadm_id'].unique()

In [62]:
len(hids)

224745

In [99]:
los=12#int(los/bucket)

In [100]:
los

12

In [102]:
for hid in tqdm(hids):
    df2=final[final['hadm_id']==hid]
    #print(hid)
    df2=df2.pivot(index='start_time',columns='nonproprietaryname',values='stop_time')
    #print(df2.shape)
    add_indices = pd.Index(range(los)).difference(df2.index)
    add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan)
    df2=pd.concat([df2, add_df])
    df2=df2.ffill()
    df2=df2.fillna(0)
    #print(df2.head())
    df2.iloc[:,1:]=df2.iloc[:,1:].sub(df2.index,0)
    df2[df2>0]=1
    df2[df2<0]=0
    #print(df2.head())
    dataDic[hid]['Med']=df2.iloc[:,1:].to_dict(orient="list")
    
    ##########COND#########
    grp=cond[cond['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
    else:
        dataDic[hid]['Cond']={'fids':list(grp['root'])}

100%|██████████| 224745/224745 [50:33<00:00, 74.08it/s] 


In [103]:
dataDic[24947999]

{'Cond': {'fids': ['I70',
   'N18',
   'E66',
   'Z68',
   'I50',
   'Z95',
   'I25',
   'Z51',
   'I50',
   'I25',
   'K22',
   'E78',
   'Z51',
   'I20',
   'Z91',
   'Z91',
   'D64',
   'E11',
   'Z51',
   'I12']},
 'Proc': {},
 'Med': {'aspirin': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'atorvastatin calcium': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'carvedilol': [0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  'clopidogrel bisulfate': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'dextrose monohydrate': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'furosemide': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
  'heparin sodium': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0],
  'insulin human': 

In [98]:
final[final['hadm_id']==24947999]

,hadm_id,nonproprietaryname,stop_time,subject_id,start_time
716318,24947999,allopurinol,12.0,10000980,0
716319,24947999,aspirin,12.0,10000980,0
716320,24947999,atorvastatin calcium,12.0,10000980,0
716321,24947999,clopidogrel bisulfate,12.0,10000980,0
716322,24947999,dextrose monohydrate,12.0,10000980,0
716323,24947999,furosemide,12.0,10000980,0
716324,24947999,heparin sodium,12.0,10000980,0
716325,24947999,insulin lispro,12.0,10000980,0
716326,24947999,ipratropium bromide,12.0,10000980,0
716327,24947999,magnesium sulfate in water,12.0,10000980,0


In [85]:
meds[meds['hadm_id']==24947999]

,subject_id,hadm_id,starttime,nonproprietaryname,start_hours_from_admit,stop_hours_from_admit,start_time,stop_time,select_time
191,10000980,24947999,2190-11-07 12:00:00,aspirin,0 days 15:03:00,1 days 23:03:00,0,24,19
189,10000980,24947999,2190-11-07 14:00:00,insulin lispro,0 days 17:03:00,0 days 20:03:00,0,1,19
195,10000980,24947999,2190-11-07 15:00:00,insulin lispro,0 days 18:03:00,1 days 23:03:00,0,24,19
238,10000980,24947999,2190-11-07 12:00:00,furosemide,0 days 15:03:00,1 days 23:03:00,0,24,19
209,10000980,24947999,2190-11-07 12:00:00,atorvastatin calcium,0 days 15:03:00,1 days 23:03:00,0,24,19
229,10000980,24947999,2190-11-07 14:00:00,dextrose monohydrate,0 days 17:03:00,1 days 23:03:00,0,24,19
224,10000980,24947999,2190-11-07 12:00:00,ipratropium bromide,0 days 15:03:00,1 days 23:03:00,0,24,19
213,10000980,24947999,2190-11-07 14:00:00,magnesium sulfate in water,0 days 17:03:00,1 days 23:03:00,0,24,19
254,10000980,24947999,2190-11-07 12:00:00,allopurinol,0 days 15:03:00,1 days 23:03:00,0,24,19
250,10000980,24947999,2190-11-07 12:00:00,clopidogrel bisulfate,0 days 15:03:00,1 days 23:03:00,0,24,19


In [316]:
sample['start_time'].max()

110

In [ ]:
maxcount=0
for hid in tqdm(hids):
    grp=cond[cond['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Cond']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Cond']={'fids':list(grp['root'])}

In [223]:
maxcount=0
for hid in tqdm(hids):
    sub=meds[meds['hadm_id']==hid]
    groups=sub.groupby('nonproprietaryname')
    for nonproprietaryname, grp in groups:
        sig=full_signal(grp)
        #print(sig)
        dataDic[hid]['Med'][nonproprietaryname]=list(sig)
    #print(dataDic)

  0%|          | 1008/344107 [02:13<12:34:39,  7.58it/s]


KeyboardInterrupt: 

In [77]:
maxcount=0
for hid in hids:
    grp=meds[meds['hadm_id']==hid]
    if(grp.shape[0]==0):
        dataDic[hid]['Med']={'fids':list(['<PAD>'])}
    else:
        if(grp.shape[0]>maxcount):
            maxcount=grp.shape[0]
        dataDic[hid]['Med']={'fids':list(grp['nonproprietaryname'])}
    #print(condDic)

In [78]:
maxcount

819

In [ ]:
def full_signal(grp):
    grp=grp.sort_values(by=['start_time'])
    val=[]
    old_lab=np.nan
    for i in range(1,los,bucket):
        start=grp[(grp['start_time']>i) & (grp['start_time']<i+bucket)]
        if(start.shape[0]>0):
            old_lab=start['val'].mean()
        val=val.append(old_lab)
    col_mean = np.nanmean(a, axis=0)
    #Find indices that you need to replace
    inds = np.where(np.isnan(val))

    #Place column means in the indices. Align the arrays using take
    val[inds] = np.take(col_mean, inds[1])

    return val

In [ ]:
maxcount=0
for hid in hids:
    sub=labs[labs['hadm_id']==hid]
    groups=sub.group_by('code')
    for code, grp in groups:
        all_mean=labs[labs['code']==code]['val'].mean()
        subject_mean=labs[(labs['code']==code) & (labs['subject_id'].isin(list(sub['subject_id'])))]['val'].mean()
        sig=full_signal(grp)
        dataDic[hid]['Lab'][code]=list(sig)

In [104]:
cond.groupby('hadm_id').size().max()

56

In [105]:
path='C:/Users/mehak/OneDrive - University of Delaware - o365/Beheshti, Rahmat - Mehak - Brennan/model/data/'

In [106]:
with open(path+'dataDic', 'wb') as fp:
    pickle.dump(dataDic, fp)

In [107]:
with open(path+'hadmDic', 'wb') as fp:
    pickle.dump(hids, fp)

In [108]:
with open(path+'medVocab', 'wb') as fp:
    pickle.dump(list(meds['nonproprietaryname'].unique()), fp)

In [83]:
with open(path+'procVocab', 'wb') as fp:
    pickle.dump(list(proc['icd_code'].unique()), fp)

In [109]:
with open(path+'condVocab', 'wb') as fp:
    pickle.dump(list(cond['root'].unique()), fp)

In [110]:
meds['nonproprietaryname'].nunique()

817

In [86]:
proc['icd_code'].nunique()

4839

In [111]:
cond['root'].nunique()

1517